In [37]:
import numpy as np
import scipy #only sparse matrices allowed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [151]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#to deal with imbalanced classes
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [152]:
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingRandomSearchCV

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [40]:
#We need to increase the categorization accuracy.

In [41]:
df_train = pd.read_csv(r"./ift3395-6390-weatherevents/train.csv")

In [42]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [43]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


In [44]:
def df_year_handler(df_train):
    #df_train is the train df loaded from our csv file
    #labels are not dealt with here, so keep them in a separate column before calling this
    #df_train2 = df_train.drop(columns=["S.No"])
    df_train2 = df_train
    #df_train2 = df_train
    #https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    #converting to string for the split
    df_train2["time"] = df_train2["time"].astype(str)
    #expand will return multiple columns with the split strings
    split_columns = df_train2["time"].str.split("",expand=True)
    year = split_columns[1] + split_columns[2] + split_columns[3] +split_columns[4]
    month = split_columns[5] + split_columns[6]
    date = split_columns[7] + split_columns[8]
    
    #converting series to df and renaming the column so we dont end up with a bunch of columns named 0
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    year = pd.DataFrame(year)
    month = pd.DataFrame(month)
    date = pd.DataFrame(date)

    year = year.rename(columns={0: "year"})
    month = month.rename(columns={0: "month"})
    date = date.rename(columns={0: "date"})

    #we need to convert the strings back to integers
    #https://datatofish.com/string-to-integer-dataframe/
    year['year'] = year['year'].astype(int)
    month['month'] = month['month'].astype(int)
    date['date'] = date['date'].astype(int)
    
    #adding these back to the original df
    df = pd.concat([df_train2, year, month, date], axis = 1)
    df = df.drop(columns=["time"])
    #df.head()
    
    return df

### I hear there are duplicates in the data

### There are samples which have the same feature values but different target values

### let's try to handle these

In [45]:
df_train

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,...,213.804184,259.846222,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,20001106,2
47756,47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,...,213.867416,263.996307,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,20010712,2
47757,47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,...,216.394852,264.192566,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,20011029,0
47758,47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,...,217.026199,265.778900,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,20020712,0


In [46]:
df_train = df_year_handler(df_train)

In [47]:
labels = df_train[["LABELS"]]

In [48]:
df_train = df_train.drop(columns=["S.No","LABELS"])

In [49]:
df_train = pd.concat([df_train, labels],axis=1)

In [50]:
df_train

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,100803.1094,...,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,2007,10,6,1
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,102553.8984,...,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,2000,9,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,102109.4844,...,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,2000,11,6,2
47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,101734.2813,...,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,2001,7,12,2
47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,101800.8516,...,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,2001,10,29,0
47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,101641.2266,...,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,2002,7,12,0


In [51]:
#Dropping raw duplicates with labels
df_temp1 = df_train.drop_duplicates()

In [52]:
df_temp1

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,100803.1094,...,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,2007,10,6,1
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,102553.8984,...,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,2000,9,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47743,23.585398,276.5625,41.815079,-5.981042,7.609122,-8.963073,2.837600,0.020747,101517.6328,101517.6406,...,3.870000e-08,302.773254,302.241974,134.770569,12433.16016,66.823296,2001,8,4,0
47744,-24.993481,242.5000,23.160349,-6.694213,1.401017,-8.668851,4.462517,0.010743,102259.7891,102259.7891,...,7.730000e-09,295.015991,293.339417,192.308746,12169.76367,64.421112,2008,10,3,0
47750,13.494133,253.7500,44.489532,-2.837747,-1.062834,-3.629961,-1.991696,0.017999,100848.6094,100848.6094,...,7.100000e-10,302.839966,301.316742,75.255516,12388.10938,66.508980,1997,6,18,0
47754,13.024772,253.4375,53.848255,10.059551,-0.746791,9.813080,1.184948,0.019498,100899.0000,100899.0000,...,2.430000e-07,302.068481,300.820374,79.624161,12387.60742,66.427940,2004,10,3,0


In [53]:
# Drop all feature duplicates, which may have different labels that confuse our model
df_temp2 = df_temp1.drop_duplicates(subset=df_temp1.columns[0:-1], keep=False)

In [54]:
df_temp2_labels = df_temp2[["LABELS"]]
df_temp2_labels

,LABELS
0,0
2,0
3,0
5,2
6,0
...,...
47742,0
47743,0
47744,0
47750,0


In [55]:
df_temp2 = df_temp2.drop(columns=["LABELS"])
df_temp2

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6
5,-24.524120,241.5625,43.424503,10.390884,-10.737404,2.822728,-11.261213,0.014424,101099.9375,101099.9375,...,264.098419,5.650000e-08,296.483215,296.177185,95.622093,12237.61719,65.247345,2001,9,19
6,-24.993481,242.1875,34.667477,-1.841543,3.988539,1.742689,6.661032,0.013957,101552.2656,101552.2656,...,264.075195,3.570000e-09,299.140747,297.297180,134.934036,12242.46875,65.431206,2001,4,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47742,23.585398,277.5000,24.418753,3.641168,6.957775,-8.155352,3.388716,0.013139,102010.6250,102012.7266,...,261.399536,8.420000e-19,297.565277,295.948151,173.441727,12256.40039,65.099129,2001,2,2
47743,23.585398,276.5625,41.815079,-5.981042,7.609122,-8.963073,2.837600,0.020747,101517.6328,101517.6406,...,265.900665,3.870000e-08,302.773254,302.241974,134.770569,12433.16016,66.823296,2001,8,4
47744,-24.993481,242.5000,23.160349,-6.694213,1.401017,-8.668851,4.462517,0.010743,102259.7891,102259.7891,...,262.539703,7.730000e-09,295.015991,293.339417,192.308746,12169.76367,64.421112,2008,10,3
47750,13.494133,253.7500,44.489532,-2.837747,-1.062834,-3.629961,-1.991696,0.017999,100848.6094,100848.6094,...,268.018402,7.100000e-10,302.839966,301.316742,75.255516,12388.10938,66.508980,1997,6,18


In [56]:
df_temp2_std = (df_temp2-df_temp2.mean())/df_temp2.std()
df_temp2_np_std = df_temp2_std.to_numpy()
df_temp2_labels_np = df_temp2_labels.to_numpy()

In [58]:
#train a model to make predictions on conflicting values
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

knn_clf_for_good_data = KNeighborsClassifier(n_neighbors=5).fit(df_temp2_np_std, df_temp2_labels_np.ravel())


In [59]:
# predicting on duplicates
# duplicates is a mask
duplicates = df_temp1.duplicated(subset=df_temp1.columns[0:-1], keep='first')
duplicates_df = df_temp1[duplicates]
duplicates_df_std = (duplicates_df-duplicates_df.mean())/duplicates_df.std()
duplicates_df_std = duplicates_df_std.drop(columns=["LABELS"])
duplicates_df_std_np = duplicates_df_std.to_numpy()

predictions = knn_clf_for_good_data.predict(duplicates_df_std_np)

In [60]:
labels_new = pd.DataFrame(data=predictions, columns=["LABELS"])

In [61]:
labels_new

,LABELS
0,0
1,0
2,2
3,0
4,0
...,...
3198,0
3199,0
3200,0
3201,0


In [62]:
duplicates_df = duplicates_df.reset_index()
duplicates_df = duplicates_df.drop(columns=["index","LABELS"])
duplicates_df

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date
0,-24.289439,241.2500,34.993114,-1.250208,1.830655,-1.164386,0.791331,0.013837,102127.5781,102127.5781,...,264.752441,3.730000e-15,299.001129,297.850922,184.584137,12322.55469,65.601128,2001,5,7
1,23.820078,276.8750,47.643150,-8.193987,-0.793653,2.222956,-2.483554,0.014301,101641.0391,101641.0391,...,267.657288,1.640000e-13,302.688385,301.619019,144.736191,12478.61816,66.431099,2010,7,5
2,-31.095176,353.4375,17.494404,5.393440,-10.987731,2.693818,-11.873419,0.007240,102210.5703,102210.5703,...,257.774780,4.920000e-09,290.675598,289.177429,185.113617,12021.61426,63.365555,2002,8,24
3,21.473272,275.3125,47.726372,-7.007339,9.557825,-7.793356,-0.364407,0.016162,101481.4141,101481.4609,...,267.678680,3.050000e-11,302.522949,301.916351,131.041611,12470.16699,66.546646,2010,7,5
4,-24.758801,242.5000,16.059599,3.491873,2.355471,0.478088,3.135037,0.007984,101822.5000,101822.5000,...,259.952759,6.140000e-12,296.630707,293.299530,155.424042,12095.59961,64.323791,2006,6,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,-20.769231,229.6875,17.247395,5.472986,-3.642907,-1.382102,-4.805233,0.012555,101502.6406,101502.6406,...,265.733765,5.370000e-17,297.768890,296.710236,130.367966,12308.53027,65.280739,2000,9,28
3199,13.494133,252.8125,53.929550,-7.672997,-4.186466,5.395471,5.290195,0.018129,100943.5156,100943.5156,...,266.613464,4.480000e-20,302.127685,298.890289,82.900589,12407.66797,65.909958,2009,7,15
3200,-21.003911,229.6875,34.676754,-9.601542,1.853052,-11.285004,4.884233,0.013464,102099.4609,102099.4609,...,264.383758,1.590000e-07,297.313141,296.051178,180.867950,12299.45996,65.126869,2010,10,19
3201,-21.003911,229.0625,23.064512,-3.096328,-3.706821,-3.422838,-1.319323,0.013747,101791.6016,101791.6016,...,266.522919,4.100000e-15,296.893219,296.203308,154.993484,12338.62500,65.235397,1996,10,3


In [63]:
final_df1 = pd.concat([duplicates_df, labels_new], axis = 1)
final_df1

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.289439,241.2500,34.993114,-1.250208,1.830655,-1.164386,0.791331,0.013837,102127.5781,102127.5781,...,3.730000e-15,299.001129,297.850922,184.584137,12322.55469,65.601128,2001,5,7,0
1,23.820078,276.8750,47.643150,-8.193987,-0.793653,2.222956,-2.483554,0.014301,101641.0391,101641.0391,...,1.640000e-13,302.688385,301.619019,144.736191,12478.61816,66.431099,2010,7,5,0
2,-31.095176,353.4375,17.494404,5.393440,-10.987731,2.693818,-11.873419,0.007240,102210.5703,102210.5703,...,4.920000e-09,290.675598,289.177429,185.113617,12021.61426,63.365555,2002,8,24,2
3,21.473272,275.3125,47.726372,-7.007339,9.557825,-7.793356,-0.364407,0.016162,101481.4141,101481.4609,...,3.050000e-11,302.522949,301.916351,131.041611,12470.16699,66.546646,2010,7,5,0
4,-24.758801,242.5000,16.059599,3.491873,2.355471,0.478088,3.135037,0.007984,101822.5000,101822.5000,...,6.140000e-12,296.630707,293.299530,155.424042,12095.59961,64.323791,2006,6,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,-20.769231,229.6875,17.247395,5.472986,-3.642907,-1.382102,-4.805233,0.012555,101502.6406,101502.6406,...,5.370000e-17,297.768890,296.710236,130.367966,12308.53027,65.280739,2000,9,28,0
3199,13.494133,252.8125,53.929550,-7.672997,-4.186466,5.395471,5.290195,0.018129,100943.5156,100943.5156,...,4.480000e-20,302.127685,298.890289,82.900589,12407.66797,65.909958,2009,7,15,0
3200,-21.003911,229.6875,34.676754,-9.601542,1.853052,-11.285004,4.884233,0.013464,102099.4609,102099.4609,...,1.590000e-07,297.313141,296.051178,180.867950,12299.45996,65.126869,2010,10,19,0
3201,-21.003911,229.0625,23.064512,-3.096328,-3.706821,-3.422838,-1.319323,0.013747,101791.6016,101791.6016,...,4.100000e-15,296.893219,296.203308,154.993484,12338.62500,65.235397,1996,10,3,0


In [64]:
df_temp2 = pd.concat([df_temp2, df_temp2_labels], axis = 1)
df_temp2

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
5,-24.524120,241.5625,43.424503,10.390884,-10.737404,2.822728,-11.261213,0.014424,101099.9375,101099.9375,...,5.650000e-08,296.483215,296.177185,95.622093,12237.61719,65.247345,2001,9,19,2
6,-24.993481,242.1875,34.667477,-1.841543,3.988539,1.742689,6.661032,0.013957,101552.2656,101552.2656,...,3.570000e-09,299.140747,297.297180,134.934036,12242.46875,65.431206,2001,4,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47742,23.585398,277.5000,24.418753,3.641168,6.957775,-8.155352,3.388716,0.013139,102010.6250,102012.7266,...,8.420000e-19,297.565277,295.948151,173.441727,12256.40039,65.099129,2001,2,2,0
47743,23.585398,276.5625,41.815079,-5.981042,7.609122,-8.963073,2.837600,0.020747,101517.6328,101517.6406,...,3.870000e-08,302.773254,302.241974,134.770569,12433.16016,66.823296,2001,8,4,0
47744,-24.993481,242.5000,23.160349,-6.694213,1.401017,-8.668851,4.462517,0.010743,102259.7891,102259.7891,...,7.730000e-09,295.015991,293.339417,192.308746,12169.76367,64.421112,2008,10,3,0
47750,13.494133,253.7500,44.489532,-2.837747,-1.062834,-3.629961,-1.991696,0.017999,100848.6094,100848.6094,...,7.100000e-10,302.839966,301.316742,75.255516,12388.10938,66.508980,1997,6,18,0


In [65]:
final_df = pd.concat([df_temp2, final_df1], ignore_index = True)

In [66]:
final_df

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
1,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
2,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
3,-24.524120,241.5625,43.424503,10.390884,-10.737404,2.822728,-11.261213,0.014424,101099.9375,101099.9375,...,5.650000e-08,296.483215,296.177185,95.622093,12237.61719,65.247345,2001,9,19,2
4,-24.993481,242.1875,34.667477,-1.841543,3.988539,1.742689,6.661032,0.013957,101552.2656,101552.2656,...,3.570000e-09,299.140747,297.297180,134.934036,12242.46875,65.431206,2001,4,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22645,-20.769231,229.6875,17.247395,5.472986,-3.642907,-1.382102,-4.805233,0.012555,101502.6406,101502.6406,...,5.370000e-17,297.768890,296.710236,130.367966,12308.53027,65.280739,2000,9,28,0
22646,13.494133,252.8125,53.929550,-7.672997,-4.186466,5.395471,5.290195,0.018129,100943.5156,100943.5156,...,4.480000e-20,302.127685,298.890289,82.900589,12407.66797,65.909958,2009,7,15,0
22647,-21.003911,229.6875,34.676754,-9.601542,1.853052,-11.285004,4.884233,0.013464,102099.4609,102099.4609,...,1.590000e-07,297.313141,296.051178,180.867950,12299.45996,65.126869,2010,10,19,0
22648,-21.003911,229.0625,23.064512,-3.096328,-3.706821,-3.422838,-1.319323,0.013747,101791.6016,101791.6016,...,4.100000e-15,296.893219,296.203308,154.993484,12338.62500,65.235397,1996,10,3,0


In [67]:
df_train = final_df

In [242]:
df_train

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
1,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
2,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
3,-24.524120,241.5625,43.424503,10.390884,-10.737404,2.822728,-11.261213,0.014424,101099.9375,101099.9375,...,5.650000e-08,296.483215,296.177185,95.622093,12237.61719,65.247345,2001,9,19,2
4,-24.993481,242.1875,34.667477,-1.841543,3.988539,1.742689,6.661032,0.013957,101552.2656,101552.2656,...,3.570000e-09,299.140747,297.297180,134.934036,12242.46875,65.431206,2001,4,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22645,-20.769231,229.6875,17.247395,5.472986,-3.642907,-1.382102,-4.805233,0.012555,101502.6406,101502.6406,...,5.370000e-17,297.768890,296.710236,130.367966,12308.53027,65.280739,2000,9,28,0
22646,13.494133,252.8125,53.929550,-7.672997,-4.186466,5.395471,5.290195,0.018129,100943.5156,100943.5156,...,4.480000e-20,302.127685,298.890289,82.900589,12407.66797,65.909958,2009,7,15,0
22647,-21.003911,229.6875,34.676754,-9.601542,1.853052,-11.285004,4.884233,0.013464,102099.4609,102099.4609,...,1.590000e-07,297.313141,296.051178,180.867950,12299.45996,65.126869,2010,10,19,0
22648,-21.003911,229.0625,23.064512,-3.096328,-3.706821,-3.422838,-1.319323,0.013747,101791.6016,101791.6016,...,4.100000e-15,296.893219,296.203308,154.993484,12338.62500,65.235397,1996,10,3,0


In [243]:
latest_labels = df_train[["LABELS"]]

In [244]:
def df_to_train_test_split(df):
    #df is the normalized or standardized df
    #labels are just the labels.
#         So we need 6760 test exmaples, in 18:4:1 ratio roughly to match the train test.<br>
#         So we need:<br>
#         1s - 294<br>
#         2s - 1176<br>
#         0s - 5290<br>
    data_with_labels = df.sample(frac=1).reset_index(drop=True)
    
    data_with_labels_0s = data_with_labels[data_with_labels["LABELS"] == 0]
    data_with_labels_1s = data_with_labels[data_with_labels["LABELS"] == 1]
    data_with_labels_2s = data_with_labels[data_with_labels["LABELS"] == 2]

    data_1_test = data_with_labels_1s.iloc[0:109]
    data_2_test = data_with_labels_2s.iloc[0:458]
    data_0_test = data_with_labels_0s.iloc[0:2827]

    data_1_train = data_with_labels_1s.iloc[109:]
    data_2_train = data_with_labels_2s.iloc[458:]
    data_0_train = data_with_labels_0s.iloc[2827:]

    test = pd.concat([data_1_test, data_2_test, data_0_test])
    #Houston, we have a problem, we need to shuffle the indices, but pandas has us covered
    #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    train = pd.concat([data_1_train, data_2_train, data_0_train])

    #Shuffling:
    #df = df.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    train = train.sample(frac=1).reset_index(drop=True)

    #Finally, we have our train and test sets.
    #Let's try to follow this post: https://github.com/rahulrrai/multinomial-logistic-regression/blob/master/multinomial-logistic-regression.ipynb
    train_np = train.to_numpy()
    test_np = test.to_numpy()

    #number of training examples
    #this is 41000 in our case
    num_of_train = train_np.shape[0]
    num_of_test = test_np.shape[0]
    #retaining just the features
    X_train = train_np[:,:-1]
    X_test = test_np[:,:-1]
    ##As we are using vector implementation adding a column in X with all the value 1
    # I'm guessing this is the bias term added to our train matrix itself
#     X_train = np.hstack((np.ones((num_of_train,1)),X_train))
#     X_test = np.hstack((np.ones((num_of_test,1)), X_test))
    #output 
    y_train = train_np[:,-1:]
    y_test = test_np[:,-1:]
    #each individual entry in y is a numpy array, let's see if this works out.
    
    return X_train, X_test, y_train, y_test

In [245]:
X_train, X_test, y_train, y_test = df_to_train_test_split(df_train)

In [246]:
def standardizer_new(X_train):
    df = pd.DataFrame(X_train)
    #df = df.drop(columns=[0])
    df = (df-df.mean())/df.std()
    X_train = df.to_numpy()
    #num_of_train = X_train.shape[0]
    #X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    return X_train

In [247]:
X_train_std = standardizer_new(X_train)
X_train_std

array([[ 0.68873718, -0.39070875,  1.39533851, ...,  0.33124424,
        -0.668571  ,  0.89319261],
       [ 1.07808046,  0.17341656,  0.56708569, ..., -0.16409156,
        -1.49610864,  1.00111937],
       [-0.79691482, -0.97008068, -0.5718272 , ..., -0.41175946,
        -0.25480217,  0.24563203],
       ...,
       [ 0.69898306, -0.39833206,  0.88015028, ..., -1.15476315,
        -0.25480217, -1.04948912],
       [ 1.06783459,  0.17341656,  0.61881638, ..., -1.15476315,
        -0.668571  , -1.04948912],
       [ 1.16004747,  0.21153313,  1.8693781 , ..., -1.15476315,
         0.15896665,  0.24563203]])

In [248]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [249]:
X_test_std = standardizer_new(X_test)

In [250]:
X_train_std.shape

(19256, 21)

In [251]:
X_test_std.shape

(3394, 21)

In [252]:
y_train.shape

(19256, 1)

In [253]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([15737,   611,  2908], dtype=int64))

In [254]:
y_test.shape

(3394, 1)

In [255]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2]), array([2827,  109,  458], dtype=int64))

In [256]:
X_train_full_std = np.concatenate((X_train_std, X_test_std), axis=0)

In [257]:
y_train_full = np.concatenate((y_train, y_test), axis=0)

In [258]:
np.savetxt("X_train_std.csv", X_train_std, delimiter=",")
np.savetxt("y_train.csv", y_train, delimiter=",")

In [259]:
np.savetxt("X_train_full_std.csv", X_train_full_std, delimiter=",")
np.savetxt("y_train_full.csv", y_train_full, delimiter=",")

In [260]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [261]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([15737,   611,  2908], dtype=int64))

In [262]:
# Combining undersampling with oversampling since that gives better results

In [263]:
rus = RandomUnderSampler(sampling_strategy={0:11000,1:611,2:2908},random_state=66)
X_temp, y_temp = rus.fit_resample(X_train, y_train)

In [264]:
np.unique(y_temp, return_counts=True)

(array([0, 1, 2]), array([11000,   611,  2908], dtype=int64))

In [265]:
smt = SMOTE(sampling_strategy={0:11000, 1:2400, 2:7000}, random_state = 66)
X_train_smt, y_train_smt = smt.fit_resample(X_temp, y_temp)

In [266]:
np.unique(y_train_smt, return_counts=True)

(array([0, 1, 2]), array([11000,  2400,  7000], dtype=int64))

In [267]:
X_train_smt.shape

(20400, 21)

In [268]:
X_train_smt_std = standardizer_new(X_train_smt)
y_train_smt = y_train_smt

In [269]:
X_train_smt_std.shape

(20400, 21)

In [270]:
X_test_std.shape

(3394, 21)

In [271]:
y_train_smt.shape

(20400,)

In [272]:
y_test.ravel().shape

(3394,)

In [273]:
X_train_full_smt_std = np.concatenate((X_train_smt_std, X_test_std), axis=0)

In [274]:
y_train_full_smt = np.concatenate((y_train_smt, y_test.ravel()), axis=0)

In [275]:
np.savetxt("X_train_full_smt_std.csv", X_train_full_smt_std, delimiter=",")
np.savetxt("y_train_full_smt.csv", y_train_full_smt, delimiter=",")

In [295]:
param_grid1 = {
    'n_estimators' : [50, 100, 200, 300],
    'criterion': ['gini','entropy'],
    #removed None from below to avoid overfitting
    'max_depth' : [3, 5, 9],
    'min_samples_split' : [2, 4, 8, 20, 0.01, 0.1, 0.2, 0.4],
    'min_samples_leaf' : [1, 2, 4, 8, 16],
    'max_features' : [None, "sqrt", "log2"],
    'bootstrap' : [True, False],
    'class_weight' : [None, "balanced", "balanced_subsample"],
}

In [296]:
rf_clf = RandomForestClassifier()
grid1 = HalvingRandomSearchCV(estimator = rf_clf, param_distributions = param_grid1, refit = True, verbose = 2, n_jobs=-1, factor = 3)
# fitting the model for grid search 
grid1.fit(X_train_smt_std, y_train_smt.ravel())

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 30
max_resources_: 20400
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 680
n_resources: 30
Fitting 5 folds for each of 680 candidates, totalling 3400 fits
----------
iter: 1
n_candidates: 227
n_resources: 90
Fitting 5 folds for each of 227 candidates, totalling 1135 fits
----------
iter: 2
n_candidates: 76
n_resources: 270
Fitting 5 folds for each of 76 candidates, totalling 380 fits
----------
iter: 3
n_candidates: 26
n_resources: 810
Fitting 5 folds for each of 26 candidates, totalling 130 fits
----------
iter: 4
n_candidates: 9
n_resources: 2430
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 5
n_candidates: 3
n_resources: 7290
Fitting 5 folds for each of 3 candidates, totalling 15 fits


HalvingRandomSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                      param_distributions={'bootstrap': [True, False],
                                           'class_weight': [None, 'balanced',
                                                            'balanced_subsample'],
                                           'criterion': ['gini', 'entropy'],
                                           'max_depth': [3, 5, 9],
                                           'max_features': [None, 'sqrt',
                                                            'log2'],
                                           'min_samples_leaf': [1, 2, 4, 8, 16],
                                           'min_samples_split': [2, 4, 8, 20,
                                                                 0.01, 0.1, 0.2,
                                                                 0.4],
                                           'n_estimators': [50, 100, 200, 300]},
                    

In [297]:
# print best parameter after tuning 
print(grid1.best_params_) 
grid_predictions = grid1.predict(X_test_std)

{'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 9, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': False}


In [298]:
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

           0       0.98      0.86      0.92      2827
           1       0.61      0.81      0.69       109
           2       0.56      0.92      0.70       458

    accuracy                           0.87      3394
   macro avg       0.72      0.86      0.77      3394
weighted avg       0.91      0.87      0.88      3394



In [299]:
print("Accuracy is:",accuracy_score(y_test, grid_predictions))
print("Balanced Accuracy is:",balanced_accuracy_score(y_test, grid_predictions))
print("F1 score is:",f1_score(y_test, grid_predictions, average = None))
print("F1 sample score is:",f1_score(y_test, grid_predictions, average = None))
print("Precision is:",precision_score(y_test, grid_predictions, average = None))
print("Recall is:",recall_score(y_test, grid_predictions, average = None))

#clf.predict_proba(X), multi_class='ovr'
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
print("ROC AUC Score is:",roc_auc_score(y_test, grid1.predict_proba(X_test_std), multi_class = 'ovr'))

Accuracy is: 0.8694755450795522
Balanced Accuracy is: 0.8628456595567
F1 score is: [0.91686996 0.69291339 0.69709544]
F1 sample score is: [0.91686996 0.69291339 0.69709544]
Precision is: [0.97641886 0.60689655 0.562249  ]
Recall is: [0.86416696 0.80733945 0.91703057]
ROC AUC Score is: 0.9728533106491081


In [300]:
grid1.best_estimator_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=9,
                       max_features='log2', min_samples_split=4)

In [301]:
rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=9,
                       max_features='log2', min_samples_split=4).fit(X_train_full_smt_std, y_train_full_smt.ravel())

In [ ]:
#predictions = rf.predict()

In [302]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [303]:
submit_df = df_year_handler(submit_df)

In [304]:
submit_df = submit_df.drop(columns=["S.No"])

In [305]:
submit_df

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date
0,-31.095176,353.1250,22.372244,-1.994936,-0.002513,-7.732825,-1.864900,0.008203,102809.6719,102809.6719,...,258.135406,4.060000e-08,290.377930,289.014862,234.634003,12023.84082,63.400764,2013,9,27
1,-31.095176,353.4375,21.536032,-2.058918,0.397359,-8.436489,-0.940275,0.007887,102816.6016,102816.6016,...,258.205994,1.670000e-08,290.377228,288.873138,234.982056,12023.83301,63.348942,2013,9,27
2,-31.095176,353.7500,20.465317,-2.128485,0.717785,-8.842417,0.152449,0.007695,102827.8594,102827.8594,...,258.242279,1.220000e-08,290.377228,288.641480,235.631378,12025.41113,63.281322,2013,9,27
3,-31.095176,354.0625,19.967148,-2.126235,1.123356,-8.773702,1.251795,0.007519,102830.5391,102830.5391,...,258.335113,3.460000e-10,290.441406,288.420074,235.608460,12026.61426,63.216827,2013,9,27
4,-31.095176,354.3750,19.598663,-2.053118,1.823581,-8.472447,2.225254,0.007293,102823.8516,102823.8516,...,258.513702,5.920000e-10,290.468903,288.392975,235.004334,12027.29004,63.200485,2013,9,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,24.054759,276.5625,51.415295,-1.095974,-8.194263,2.484773,-10.520496,0.020928,100743.8438,100743.8438,...,268.880676,6.870000e-15,302.576569,302.226227,66.832077,12467.79492,66.832077,2013,9,16
7316,24.054759,276.8750,52.377407,-0.265653,-8.730537,3.783044,-10.748092,0.021389,100703.5313,100703.5313,...,269.085083,9.240000e-10,302.583557,302.131012,66.827492,12466.41309,66.827492,2013,9,16
7317,24.054759,277.1875,54.639217,0.775797,-9.646189,5.087689,-10.786784,0.021745,100673.9844,100673.9844,...,269.305939,5.870000e-08,302.646820,302.032715,66.812981,12467.56934,66.812981,2013,9,16
7318,24.054759,277.5000,56.121231,1.813888,-10.849813,6.442380,-10.859090,0.021840,100647.3359,100647.3359,...,269.616638,8.810000e-08,302.646820,302.014435,66.808281,12468.06934,66.808273,2013,9,16


In [306]:
def standardizer_new(X_train):
    df = pd.DataFrame(X_train)
    #df = df.drop(columns=[0])
    df = (df-df.mean())/df.std()
    X_train = df.to_numpy()
    #num_of_train = X_train.shape[0]
    #X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    return X_train

In [307]:
X_submit = standardizer_new(submit_df)

In [308]:
X_submit.shape

(7320, 21)

In [309]:
predictions_submit = rf.predict(X_submit)

In [310]:
predictions_df = pd.DataFrame(data=predictions_submit,columns=["LABELS"]).astype(int)
#https://stackoverflow.com/questions/18022845/pandas-index-column-title-or-name
predictions_df.index.name = "S.No"
predictions_df.to_csv(r"./ift3395-6390-weatherevents/submission_SMOTE_with_prior_undersampling_no_dups.csv")